Note that MIR is mortality rate divided by incidence rate. https://wonder.cdc.gov/wonder/help/CancerMIR-v2011.html#About-MIRR
$$MIR = \frac{Mortality Rate}{Incidence Rate}$$

A cancer mortality rate is the number of deaths, with cancer as the underlying cause of death, occurring in a specified population during a year.
$$Mortality Rate = \frac{Cancer Deaths}{Population} \times 100,000$$

A cancer incidence rate is the number of new cancers of a specific site/type occurring in a specified population during a year, usually expressed as the number of cancers per 100,000 population at risk.
$$Incidence Rate = \frac{New Cancers}{Population} \times 100,000$$
$\newline$

Notes for the report:
1. Break code into at least 2 Python files.
2. Test the visualization with the numbers we calculated. (Use a smaller subset to achieve)

In [1]:
import pandas as pd
import geopandas as gpd
import altair as alt
from utilities import Utils
import matplotlib.pyplot as plt

by_age = pd.read_csv("C:\\Users\\pxu11\\OneDrive - UW\\2022 Winter\\CSE 163 A\\cse163-cancer-research\\data\\USCS-1999-2018-ASCII\\BYAGE.TXT", delimiter="|", low_memory=False)
by_area = pd.read_csv("C:\\Users\\pxu11\\OneDrive - UW\\2022 Winter\\CSE 163 A\\cse163-cancer-research\\data\\USCS-1999-2018-ASCII\\BYAREA.TXT", delimiter="|", low_memory=False)
by_county = pd.read_csv("C:\\Users\\pxu11\\OneDrive - UW\\2022 Winter\\CSE 163 A\\cse163-cancer-research\\data\\USCS-1999-2018-ASCII\\BYAREA_COUNTY.TXT", delimiter="|", low_memory=False)
counties = gpd.read_file("C:\\Users\\pxu11\\OneDrive - UW\\2022 Winter\\CSE 163 A\\cse163-cancer-research\\data\\2020_us_county_shp\\cb_2020_us_county_20m.shp")

In [3]:
by_county.head(3)

,STATE,AREA,AGE_ADJUSTED_CI_LOWER,AGE_ADJUSTED_CI_UPPER,AGE_ADJUSTED_RATE,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,YEAR,CRUDE_CI_LOWER,CRUDE_CI_UPPER,CRUDE_RATE
0,AK,AK: Aleutians East Borough (02013) - 1994+,~,~,~,~,Incidence,5488,All Races,Female,All Cancer Sites Combined,2014-2018,~,~,~
1,AK,AK: Aleutians East Borough (02013) - 1994+,~,~,~,~,Mortality,5488,All Races,Female,All Cancer Sites Combined,2014-2018,~,~,~
2,AK,AK: Aleutians East Borough (02013) - 1994+,~,~,~,~,Incidence,5488,All Races,Female,Brain and Other Nervous System,2014-2018,~,~,~


In [70]:
def strip_geo_id(area):
    # elements = area.split("(")
    geoid = area[area.index("("): area.index(")") + 1]
    return geoid


def clean_join_shp(by_county, counties):
    by_county.loc[:, "geoid"] = by_county.loc[:, "AREA"] \
                                            .apply(strip_geo_id)
    by_county = by_county.astype({"geoid": "float32"})
    counties = counties.astype({"GEOID": "float32"})
    prepared_shp = counties.merge(by_county, left_on="GEOID",
                                  right_on="geoid", how="left")
    print("by_county: ", by_county["geoid"].sort())
    print("counties: ", counties["GEOID"].sort())
    return prepared_shp


def get_mir(data, on, rate_col):
    data = data.astype({rate_col: 'float32'})
    mortality = data[data["EVENT_TYPE"] == "Mortality"]
    incidence = data[data["EVENT_TYPE"] == "Incidence"]
    joined = pd.merge(mortality, incidence, how='inner', on=on)
    joined["MIR"] = joined[rate_col + "_x"] / joined[rate_col + "_y"]
    return joined


def remove_rows(data, chars):
    check = data != chars[0]  # get base filter
    for c in chars:
        check = check & (data != c)
    return data[check.all(1)]


def prepare_shp(by_county, counties):
    by_county_c = by_county[["AREA", "RACE", "SITE", "YEAR", "EVENT_TYPE",
                             "AGE_ADJUSTED_RATE", "SEX"]].copy()
    counties_c = counties[["GEOID", "NAMELSAD", "STUSPS", "STATE_NAME",
                           "geometry"]].copy()
    by_county_c = Utils.remove_rows(data=by_county_c,
                                    chars=['+', '~', '.', '-'])
    by_county_c = Utils.get_mir(data=by_county_c, 
                                on=['AREA', 'RACE', 'SEX', 'SITE', 'YEAR'],
                                rate_col='AGE_ADJUSTED_RATE')
    prepared_shp = Utils.clean_join_shp(by_county=by_county_c,
                                        counties=counties_c)
    return prepared_shp


prepared_shp = prepare_shp(by_county, counties)
print(len(prepared_shp))
no_ak = prepared_shp["STUSPS"] != "AK"
no_hi = prepared_shp["STUSPS"] != "HI"
races = prepared_shp["RACE"] == "Asian/Pacific Islander"
sites = prepared_shp["SITE"] == "Colon and Rectum"
sexes = prepared_shp["SEX"] == "Male and Female"

# alt \
#     .Chart(result[0][no_ak & no_hi & sexes & sites & races]) \
#     .mark_geoshape().encode(
#     color='MIR:Q'
# )

123613


In [63]:
by_county["RACE"].unique()

array(['All Races', 'American Indian/Alaska Native',
       'Asian/Pacific Islander', 'Black', 'Hispanic', 'White'],
      dtype=object)

In [31]:
def strip_county_name(area):
    elements = area.split(":")
    county = elements[0] +" " + elements[1].split("(")[0].strip()
    return county

by_county_c = by_county[["AREA", "RACE", "SITE", "YEAR", "EVENT_TYPE",
                         "AGE_ADJUSTED_RATE", "SEX", "STATE"]].copy()
by_county_c = Utils.remove_rows(data=by_county_c, chars=['+', '~', '.', '-'])
# by_county_c[(by_county_c["STATE"] == "KS") & \
#             (by_county_c["AREA"] == "KS: Unknown (20999)")]# & (by_county_c["EVENT_TYPE"] == "Incidence")]


print(by_county_c["YEAR"].unique())
# Somehow get.mir has erased Kansas and Nevada
state = by_county_c["STATE"] == "KS"
# len = 2037
# by_county_c[state].groupby(['AREA', 'RACE', 'SEX', 'SITE', 'YEAR'])["AGE_ADJUSTED_RATE"].min()
area = by_county_c["AREA"] == "KS: Allen County (20001)"
race = by_county_c["RACE"] == "All Races"
site = by_county_c["SITE"] == "All Cancer Sites Combined"
sex = by_county_c["SEX"] == "Female"
year = by_county_c["YEAR"] == "2014-2018"
by_county_c[state & year]


#by_county_c = Utils.get_mir(data=by_county_c, 
#                          on=['AREA', 'RACE', 'SEX', 'SITE', 'YEAR'],
#                          rate_col='AGE_ADJUSTED_RATE')
# by_county_c[by_county_c["STATE_x"] == "NE"]

#counties_c = counties[["NAMELSAD", "STUSPS", "geometry"]].copy()
#by_county_c.loc[:, "county"] = by_county_c.loc[:, "AREA"].apply(strip_county_name)
#counties_c.loc[:, "county"] = counties_c.loc[:, "STUSPS"] + " " + counties_c.loc[:, "NAMELSAD"]

#by_county_c = by_county_c.astype({"county": "str"}, copy=True)
#counties_c = counties_c.astype({"county": "str"}, copy=True)
#test_gdf = counties_c.merge(by_county_c, left_on="county", right_on="county",
#                         how="left")

# test_gdf[test_gdf["county"].notna()]

#no_ak = test_gdf["STUSPS"] != "AK"
#no_hi = test_gdf["STUSPS"] != "HI"
#all_races = test_gdf["RACE"] == "All Races"
#all_cancer_sites = test_gdf["SITE"] == "All Cancer Sites Combined"
#both_sexes = test_gdf["SEX"] == "Male and Female"
#mortality = test_gdf["EVENT_TYPE"] == "Mortality"
#test_plot = test_gdf[all_races & all_cancer_sites & both_sexes & mortality & no_ak & no_hi]
#test_plot.plot(figsize=(250, 500))

['2014-2018']


,AREA,RACE,SITE,YEAR,EVENT_TYPE,AGE_ADJUSTED_RATE,SEX,STATE
796158,KS: Allen County (20001),All Races,All Cancer Sites Combined,2014-2018,Mortality,128.1,Female,KS
796171,KS: Allen County (20001),All Races,Lung and Bronchus,2014-2018,Mortality,28.6,Female,KS
796182,KS: Allen County (20001),All Races,All Cancer Sites Combined,2014-2018,Mortality,232.7,Male,KS
796192,KS: Allen County (20001),All Races,Lung and Bronchus,2014-2018,Mortality,61.2,Male,KS
796205,KS: Allen County (20001),All Races,All Cancer Sites Combined,2014-2018,Mortality,169.9,Male and Female,KS
...,...,...,...,...,...,...,...,...
842590,KS: Wyandotte County (20209),White,Oral Cavity and Pharynx,2014-2018,Mortality,2.9,Male and Female,KS
842592,KS: Wyandotte County (20209),White,Pancreas,2014-2018,Mortality,11.9,Male and Female,KS
842593,KS: Wyandotte County (20209),White,Prostate,2014-2018,Mortality,21.4,Male and Female,KS
842594,KS: Wyandotte County (20209),White,Stomach,2014-2018,Mortality,3.8,Male and Female,KS


In [8]:
state = by_county_c["STATE_x"] == "NV"
site = by_county_c["SITE"] == "All Cancer Sites Combined"
by_county_c[state]

,AREA,RACE,SITE,YEAR,EVENT_TYPE_x,AGE_ADJUSTED_RATE_x,SEX,STATE_x,EVENT_TYPE_y,AGE_ADJUSTED_RATE_y,STATE_y,MIR,county


In [30]:
state = test_gdf["STUSPS"] == "KS"
slice = ["EVENT_TYPE", "county"]
race = test_gdf["RACE"] == "White"

test_gdf[state]

,NAMELSAD,STUSPS,geometry,county,AREA,RACE,SITE,YEAR,EVENT_TYPE_x,AGE_ADJUSTED_RATE_x,SEX,EVENT_TYPE_y,AGE_ADJUSTED_RATE_y,MIR
785,Sumner County,KS,"POLYGON ((-97.80760 37.47418, -97.15333 37.475...",KS Sumner County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
915,Hamilton County,KS,"POLYGON ((-102.04463 38.26241, -101.56748 38.2...",KS Hamilton County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2379,Marshall County,KS,"POLYGON ((-96.80620 39.82754, -96.80577 40.001...",KS Marshall County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3680,Morris County,KS,"POLYGON ((-96.93029 38.60936, -96.92723 38.812...",KS Morris County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3681,Stafford County,KS,"POLYGON ((-99.02214 38.00024, -98.91314 38.000...",KS Stafford County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116299,Leavenworth County,KS,"POLYGON ((-95.18710 39.04411, -95.18013 39.128...",KS Leavenworth County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120285,Crawford County,KS,"POLYGON ((-95.08801 37.67452, -94.61787 37.673...",KS Crawford County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121933,Gray County,KS,"POLYGON ((-100.66417 38.00253, -100.22661 38.0...",KS Gray County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122198,Elk County,KS,"POLYGON ((-96.52530 37.60701, -95.96100 37.603...",KS Elk County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
counties.head(5)

In [ ]:
by_area.head(5)